## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Padang,ID,-0.9492,100.3543,77.49,87,99,3.74,overcast clouds
1,1,Pravia,ES,43.4906,-6.1508,69.22,88,75,2.30,broken clouds
2,2,Vaini,TO,-21.2000,-175.2000,73.56,94,75,8.05,broken clouds
3,3,Aklavik,CA,68.2191,-135.0107,41.00,93,100,3.44,mist
4,4,Paka,MY,4.6374,103.4368,78.73,89,100,0.00,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
prefferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Padang,ID,-0.9492,100.3543,77.49,87,99,3.74,overcast clouds
4,4,Paka,MY,4.6374,103.4368,78.73,89,100,0.00,overcast clouds
20,20,Thinadhoo,MV,0.5333,72.9333,81.59,68,12,2.82,few clouds
24,24,Eydhafushi,MV,5.1033,73.0708,80.98,73,13,6.44,few clouds
32,32,Pandan,PH,14.0453,124.1694,80.55,83,49,7.27,scattered clouds
34,34,La Asuncion,VE,11.0333,-63.8628,85.23,68,57,9.35,broken clouds
38,38,Ajdabiya,LY,30.7554,20.2263,78.76,58,0,11.99,clear sky
45,45,Umred,IN,20.8500,79.3333,77.29,93,100,7.70,heavy intensity rain
46,46,Banjar,ID,-8.1900,114.9675,76.35,88,53,6.22,broken clouds
54,54,Otukpo,NG,7.1913,8.1329,79.70,84,90,4.74,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
prefferred_cities_df.count()

City ID                223
City                   223
Country                223
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefferred_cities_df.dropna()
clean_df.count()

City ID                223
City                   223
Country                223
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Padang,ID,77.49,overcast clouds,-0.9492,100.3543,
4,Paka,MY,78.73,overcast clouds,4.6374,103.4368,
20,Thinadhoo,MV,81.59,few clouds,0.5333,72.9333,
24,Eydhafushi,MV,80.98,few clouds,5.1033,73.0708,
32,Pandan,PH,80.55,scattered clouds,14.0453,124.1694,
34,La Asuncion,VE,85.23,broken clouds,11.0333,-63.8628,
38,Ajdabiya,LY,78.76,clear sky,30.7554,20.2263,
45,Umred,IN,77.29,heavy intensity rain,20.8500,79.3333,
46,Banjar,ID,76.35,broken clouds,-8.1900,114.9675,
54,Otukpo,NG,79.70,overcast clouds,7.1913,8.1329,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Padang,ID,77.49,overcast clouds,-0.9492,100.3543,Mercure Padang
4,Paka,MY,78.73,overcast clouds,4.6374,103.4368,RUMBIA RESORT VILLA Paka Terengganu
24,Eydhafushi,MV,80.98,few clouds,5.1033,73.0708,Soneva Fushi
32,Pandan,PH,80.55,scattered clouds,14.0453,124.1694,Bonifacio Lodge
34,La Asuncion,VE,85.23,broken clouds,11.0333,-63.8628,Colinas de Matasiete
38,Ajdabiya,LY,78.76,clear sky,30.7554,20.2263,منزلة الحاج سالم خليفة العيساويه الزواوي
45,Umred,IN,77.29,heavy intensity rain,20.8500,79.3333,"Dr.Babasaheb Ambedkar Goverment Boys Hostel ,U..."
46,Banjar,ID,76.35,broken clouds,-8.1900,114.9675,The Hamsa Resort
54,Otukpo,NG,79.70,overcast clouds,7.1913,8.1329,Queen s Hotel
60,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weater</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))